In [1]:
-- Add media type handler for `text/html` requests
CREATE DOMAIN "text/html" AS TEXT;

CREATE DOMAIN

Total execution time: 00:00:00.001

In [2]:
-- Sanitize text to replace characters with HTML entities
CREATE OR REPLACE FUNCTION ft.sanitize_html(text) RETURNS text AS $$
  SELECT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE($1, '&', '&amp;'), '"', '&quot;'),'>', '&gt;'),'<', '&lt;'), '''', '&apos;')
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.003

In [7]:
-- Format all cars as HTML cards
CREATE OR REPLACE FUNCTION ft.html_cards(ft.vehicle) RETURNS text AS $$
  SELECT FORMAT($html$
    <div class="card">
      <div class="card-body">
        <h5 class="card-title">%2$s</h5>
        <p class="card-text text-truncate">%3$s</p>
      </div>
    </div>
  $html$,
  $1.id,
  ft.sanitize_html($1.name),
  $1.data
  );
$$ language sql stable;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [9]:
-- Create HTML to display all notes
CREATE OR REPLACE FUNCTION ft.html_all_vehicle() RETURNS text AS $$
  SELECT COALESCE(
    '<div class="card-columns">'
      || string_agg(ft.html_cards(n), '' ORDER BY n.id) ||
    '</div>',
    '<p class="">No notes.</p>'
  )
  FROM ft.vehicle n;
$$ language sql;


CREATE FUNCTION

Total execution time: 00:00:00.002

In [ ]:

-- Generate page to display notes
CREATE OR REPLACE FUNCTION ft.index() RETURNS "text/html" AS $$
  SELECT $html$
    <!DOCTYPE html>
    <html lang="en">
    <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>Note Taking App</title>
      <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    </head>

    <body>
      <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <a class="navbar-brand" href="/rpc/index">Full stack postgres vehicle app</a>
        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNav"
          aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
          <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse" id="navbarNav">
          <ul class="navbar-nav">
            <li class="nav-item active">
              <a class="nav-link" href="/rpc/index">Notes</a>
            </li>
            <li class="nav-item">
              <a class="nav-link" href="/rpc/new">Create Note</a>
            </li>
          </ul>
        </div>
      </nav>

      <div class="container mt-4">
        <h2>Notes</h2>
        $html$
          || ft.html_all_vehicle() ||
        $html$
      </div>

      <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
      <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
      <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
    </body>
    </html>
  $html$
$$ language sql;
